In [2]:
import rope.base.project
from rope.refactor.extract import ExtractMethod


In [36]:
myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('test.py')
extract_span = r'''f"test [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"'''
print(extract_span)
start, end = myresource.read().find(extract_span), myresource.read().find(extract_span) + len(extract_span)
print(start, end)

extractor = ExtractMethod(myproject, myresource, start, end)
change_set = extractor.get_changes("helper")
for change in change_set.changes:
    print(change.get_description())

f"test [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
148 220
--- a/test.py
+++ b/test.py
@@ -7,5 +7,8 @@
     a = 1
     b = 2 * a
     c = a * 2 + b * 3
+    helper()
+    print(b, c)
+
+def helper():
     f"test [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
-    print(b, c)



In [37]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = r"""edit_sweep_comment(
                review_message + "\n\nSuccess! 🚀",
                4,
                pr_message=(
                    f"## Here's the PR! [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
                ),
                done=True,
            )"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))

SyntaxError: f-string: invalid syntax (<string>, line 5)